![](images/CausalGraphLearning.png)
%%HTML
<script src="require.js"></script>

# DAGs with NOTEATS Algorithm

In this lecture ,we're going to have a detailable review on DAGs with NOTEARS papre 🔎

## Abstract

Estimating the structure of directed acyclic graphs (DAGs, also known as Bayesian
networks) is a challenging problem since the search space of DAGs is combinatorial
and scales superexponentially with the number of nodes. Existing approaches
rely on various local heuristics for enforcing the acyclicity constraint. In this
paper, we introduce a fundamentally different strategy: we formulate the structure
learning problem as a purely continuous optimization problem over real matrices
that avoids this combinatorial constraint entirely. This is achieved by a novel
characterization of acyclicity that is not only smooth but also exact. The resulting
problem can be efficiently solved by standard numerical algorithms, which also
makes implementation effortless. The proposed method outperforms existing
ones, without imposing any structural assumptions on the graph such as bounded
treewidth or in-degree.

# Learning Directed Acyclic Graphs with Continuous Optimization

Learning directed acyclic graphs (DAGs) from data is an NP-hard problem [8, 11], owing mainly to the combinatorial acyclicity constraint that is difficult to enforce efficiently. At the same time, DAGs are popular models in practice, with applications in biology [33], genetics [49], machine learning [22], and causal inference [42]. For this reason, the development of new methods for learning DAGs remains a central challenge in machine learning and statistics.

## Continuous Optimization Approach

In this paper, we propose a new approach for score-based learning of DAGs by converting the traditional combinatorial optimization problem (left) into a continuous program (right):

$$
\min_{W\in\mathbb{R}^{d\times d}} F(W) \text{ subject to } G(W) \in \text{DAGs} \quad \Leftrightarrow \quad \min_{W\in\mathbb{R}^{d\times d}} F(W) \text{ subject to } h(W)=0
$$

where:
- $G(W)$ is the $d$-node graph induced by the weighted adjacency matrix $W$
- $F: \mathbb{R}^{d\times d} \to \mathbb{R}$ is a score function (see Section 2.1 for details)
- $h: \mathbb{R}^{d\times d} \to \mathbb{R}$ is our key technical device: a smooth function over real matrices whose level set at zero exactly characterizes acyclic graphs

## Advantages

Although the two problems are equivalent, the continuous program on the right:
- Eliminates the need for specialized algorithms tailored to search over the combinatorial space of DAGs
- Allows leveraging standard numerical algorithms for constrained problems
- Makes implementation particularly easy, not requiring any knowledge about graphical models

This is similar in spirit to the situation for undirected graphical models, where the formulation of a continuous log-det program [4] sparked remarkable advances in structure learning for undirected graphs (Section 2.2).


## Contributions

The main thrust of this work is to re-formulate score-based learning of DAGs so that standard smooth optimization schemes such as L-BFGS [28] can be leveraged. To accomplish this, we make the following specific contributions:

- **Explicit smooth acyclicity constraint**:  
  We explicitly construct a smooth function over ℝ<sup>d×d</sup> with computable derivatives that encodes the acyclicity constraint. This allows us to replace the combinatorial constraint *G ∈ D* with a smooth equality constraint.

- **Equality-constrained program for DAG learning**:  
  We develop an equality-constrained program for simultaneously estimating the structure and parameters of a sparse DAG from possibly high-dimensional data, and show how standard numerical solvers can be used to find stationary points.

- **Empirical effectiveness**:  
  We demonstrate the effectiveness of the resulting method in empirical evaluations against existing state-of-the-art approaches. (See Figure 1 for a quick illustration and Section 5 for details.)

- **Comparison with global minimizer**:  
  We compare our output to the exact global minimizer [12], and show that our method attains scores that are comparable to the globally optimal score in practice, although our methods are only guaranteed to find stationary points.



# Background

The basic DAG learning problem is formulated as follows:  
Let $X \in \mathbb{R}^{n \times d}$ be a data matrix consisting of $n$ i.i.d. observations of the random vector $X = (X_1, \dots, X_d)$. Let $\mathcal{D}$ denote the (discrete) space of DAGs $G = (V, E)$ on $d$ nodes. Given $X$, we seek to learn a DAG $G \in \mathcal{D}$ (a *Bayesian network*) for the joint distribution $P(X)$ [22, 42].  

We model $X$ via a **structural equation model (SEM)** defined by a weighted adjacency matrix $W \in \mathbb{R}^{d \times d}$. Instead of operating on the discrete space $\mathcal{D}$, we work in the continuous space $\mathbb{R}^{d \times d}$.

---

## 2.1 Score Functions and SEM

Any $W \in \mathbb{R}^{d \times d}$ defines a graph on $d$ nodes via:  
$$
A(W)_{ij} = \begin{cases} 
1 & \text{if } w_{ij} \neq 0, \\
0 & \text{otherwise},
\end{cases}
$$  
where $A(W)$ is the adjacency matrix of the directed graph $G(W)$. We abuse notation by treating $W$ itself as a weighted graph.

### Linear SEM
For $W = [w_1 \mid \cdots \mid w_d]$, the linear SEM is:  
$$
X_j = w_j^\top X + z_j \quad (j = 1, \dots, d),
$$  
where:
- $X = (X_1, \dots, X_d)$ is a random vector,
- $z = (z_1, \dots, z_d)$ is a noise vector (not assumed Gaussian).

### Generalized Linear Model (GLM) Extension
For non-linear dependencies, we model:  
$$
\mathbb{E}[X_j \mid X_{\text{pa}(X_j)}] = f(w_j^\top X),
$$  
where $f$ is a link function (e.g., logistic for $X_j \in \{0, 1\}$).

### Focus: Linear SEM + Least Squares
This paper focuses on:
1. **Linear SEMs**,
2. **Least-squares loss**:  
$$
\ell(W; X) = \frac{1}{2n} \|X - XW\|_F^2,
$$  
where $\|\cdot\|_F$ is the Frobenius norm.

## 2.2 Previous Work

Traditional score-based learning optimizes a discrete score $Q: \mathcal{D} \to \mathbb{R}$ over the set of DAGs $\mathcal{D}$. This differs from our score $F(W)$ defined on $\mathbb{R}^{d \times d}$. The combinatorial optimization problem is:

$$
\min_{G} Q(G) \quad \text{subject to} \quad G \in \mathcal{D}
$$

**Key Score Functions**:
- BDe(u) [20]
- BGe [23] 
- BIC [10]
- MDL [6]

### Computational Challenges
- Problem (4) is **NP-hard** [8, 11] due to the nonconvex, combinatorial nature.
- Acyclicity constraint leads to superexponential growth in possible structures with $d$ [32].

### Existing Approaches
1. **Global Optimality Methods** (for small problems):
   - [12, 13, 29, 39, 40, 47]

2. **Approximate Algorithms**:
   - *Order search*: Trades acyclicity for $d!$ ordering search [30, 34–36, 43]
   - *Greedy search*: Explicit acyclicity checks per edge addition [9, 20, 31]  
   - *Coordinate descent*: [2, 16, 18]

3. **Alternative Methods**:
   - Constraint-based [41, 42]
   - Hybrid [17, 44]  
   - Bayesian [14, 27, 51]

### Comparison to Undirected Graphs
Undirected graph learning (Markov networks) saw breakthroughs when reformulated as:

$$
\text{Convex program over real, symmetric matrices} \quad \text{[4, 48]}
$$

**Why It Worked**:
- Closed-form, tractable program enabled optimization techniques [15, 21, 37].

**DAG Learning Gap**:
- No analogous reformulation exists for (4) due to acyclicity intractability.

**Our Goal**:
Propose a **continuous program** for DAG learning via a *smooth acyclicity characterization* (introduced next).